---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # Definately need read 
    # https://www.coursera.org/learn/python-data-analysis/programming/Il9Fx/assignment-4-submission/discussions/threads/1Fkg-ryCEeaIRw7T1E5tHA?page=1
    # when applicable in 2 means that if the RegionName doesn't contain parentheses, 
    # the item should be included as is, only '\n' should be removed.
    # We should expect 517 records in all, might miss 4 records! 
    # Those records will be your nightmare: 
    # Massachusetts	The Colleges of Worcester Consortium:
    # Massachusetts	The Five College Region of Western Massachusetts:
    # Minnesota	Faribault, South Central College
    # Minnesota	North Mankato, South Central College
    
    #https://stackoverflow.com/questions/41386443/create-pandas-dataframe-from-txt-file-with-specific-pattern
    #http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html

#    df = pd.pandas.read_table('university_towns.txt', header=None)
#    df.columns = ['text']
    # (.*?) lazy mode, as few as possbile. e.g return Auburn
    # (.*) greedy mode, as much as possible, e.g return Auburn (Auburn University)
    # Boolean indexing: df.loc[boolean, 'text']
    # df['State'] = df.loc[df.text.str.contains('[edit]', regex=False), 'text'].str.extract(r'(.*?)\[edit\]', expand=False)
    # df['RegionName'] = df.loc[df.State.isnull(), 'text'].str.extract(r'(.*?)\s*[\(\[]+.*[\n]*', expand=False)
    # df.State = df.State.ffill()
    # df = df.dropna()
    #df = df[[1, 2]]

# This also not work, because situation is more complex    
#    df['RegionName'] = df['text'].apply(lambda x: x.split('(')[0].strip() if x.count('(') > 0 else np.NaN)
#    df['State'] = df['text'].apply(lambda x: x.split('[')[0].strip() if x.count('[') > 0 else np.NaN).fillna(method="ffill")
#    df = df.dropna().drop('text', axis=1).reindex(
#    columns=['State', 'RegionName']).reset_index(drop=True)

# Finally, this pesudo works
#https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/hxV9SduzEea9bRJuhhP9Rg
    data = []
    state = None
    region = []
    with open('university_towns.txt') as file:
        for line in file:
    #        thisLine = line[:-1]
            if '[ed' in line: 
                state = line[:line.index('[')]
            else:
                if '(' in line:
                    region = line[:line.index('(')-1]
                else:
                    region = line[:-1]
                data.append([state,region])

    df = pd.DataFrame(data,columns = ['State','RegionName'])
    return df

In [4]:
get_list_of_university_towns().iloc[[179, 184, 217, 223]]

State                                         RegionName
179  Massachusetts              The Colleges of Worcester Consortium:
184  Massachusetts  The Five College Region of Western Massachusetts:
217      Minnesota                   Faribault, South Central College
223      Minnesota               North Mankato, South Central College

In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdplev = pd.ExcelFile('gdplev.xls')
    #only look at GDP data from the first quarter of 2000 onward
    gdplev = gdplev.parse("Sheet1", skiprows=219)

    gdplev = gdplev[[4, 5]]
    gdplev.columns = ['YearAndQuarter','GDP']
    
    for i in range(2, len(gdplev)):
        if (gdplev.iloc[i-2][1] > gdplev.iloc[i-1][1]) and (gdplev.iloc[i-1][1] > gdplev.iloc[i][1]):
            return gdplev.iloc[i-2][0]

In [6]:
get_recession_start()

'2008q3'

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdplev = pd.ExcelFile('gdplev.xls')
    #only look at GDP data from the first quarter of 2000 onward
    gdplev = gdplev.parse("Sheet1", skiprows=219)
    
    gdplev = gdplev[[4, 5]]
    gdplev.columns = ['YearAndQuarter','GDP']
    
    start = get_recession_start()
    start_index = gdplev[gdplev['YearAndQuarter'] == start].index.tolist()[0]
    gdplev=gdplev.iloc[start_index:]
    
    for i in range(2, len(gdplev)):
        if (gdplev.iloc[i-2][1] < gdplev.iloc[i-1][1]) and (gdplev.iloc[i-1][1] < gdplev.iloc[i][1]):
            return gdplev.iloc[i][0]

In [8]:
get_recession_end()

'2009q4'

In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdplev = pd.ExcelFile('gdplev.xls')
    #only look at GDP data from the first quarter of 2000 onward
    gdplev = gdplev.parse("Sheet1", skiprows=219)
    
    gdplev = gdplev[[4, 5]]
    gdplev.columns = ['YearAndQuarter','GDP']
    start = get_recession_start()
    start_index = gdplev[gdplev['YearAndQuarter'] == start].index.tolist()[0]
    end = get_recession_end()
    end_index = gdplev[gdplev['YearAndQuarter'] == end].index.tolist()[0]
    
    gdplev=gdplev.iloc[start_index:end_index+1]
    bottom = gdplev['GDP'].min()
    bottom_index = gdplev[gdplev['GDP'] == bottom].index.tolist()[0]-start_index
    return gdplev.iloc[bottom_index]['YearAndQuarter']

In [10]:
get_recession_bottom()

'2009q2'

In [11]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    data = pd.read_csv('City_Zhvi_AllHomes.csv')
    data.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'],axis=1,inplace=1)
    data['State'] = data['State'].map(states)
    
    data.set_index(['State','RegionName'],inplace=True)
    col = list(data.columns)
    col = col[0:45]
    data.drop(col,axis=1,inplace=True)
    
    #Arrange month by quarter
    MonthByQuarter = [list(data.columns)[x:x+3] for x in range(0, len(list(data.columns)), 3)]
    
    # YearAndQuarter
    Years = list(range(2000,2017))
    Quarters = ['q1','q2','q3','q4']
    YearAndQuarter = []
    for y in Years:
        for q in Quarters:
            YearAndQuarter.append((str(y)+q))        
    YearAndQuarter = YearAndQuarter[:67]
    
    #need explore more about the zip functionality. 
    for col,quarter in zip(YearAndQuarter,MonthByQuarter):
        data[col] = data[quarter].mean(axis=1)
    data = data[YearAndQuarter]
    
    return data

In [12]:
convert_housing_data_to_quarters()

2000q1        2000q2  \
State                RegionName                                        
New York             New York                      NaN           NaN   
California           Los Angeles          2.070667e+05  2.144667e+05   
Illinois             Chicago              1.384000e+05  1.436333e+05   
Pennsylvania         Philadelphia         5.300000e+04  5.363333e+04   
Arizona              Phoenix              1.118333e+05  1.143667e+05   
Nevada               Las Vegas            1.326000e+05  1.343667e+05   
California           San Diego            2.229000e+05  2.343667e+05   
Texas                Dallas               8.446667e+04  8.386667e+04   
California           San Jose             3.742667e+05  4.065667e+05   
Florida              Jacksonville         8.860000e+04  8.970000e+04   
California           San Francisco        4.305000e+05  4.644667e+05   
Texas                Austin               1.429667e+05  1.452667e+05   
Michigan             Detroit              6.620000e+04  6.830000e+04   
Ohio                 Columbus             9.436667e+04  9.583333e+04   
Tennessee            Memphis              7.250000e+04  7.320000e+04   
North Carolina       Charlotte            1.269333e+05  1.283667e+05   
Texas                El Paso              7.626667e+04  7.686667e+04   
Massachusetts        Boston               2.069333e+05  2.191667e+05   
Washington           Seattle              2.486000e+05  2.556000e+05   
Maryland             Baltimore                     NaN           NaN   
Colorado             Denver               1.622333e+05  1.678333e+05   
District of Columbia Washington           1.377667e+05  1.442000e+05   
Tennessee            Nashville            1.138333e+05  1.152667e+05   
Wisconsin            Milwaukee            7.803333e+04  7.906667e+04   
Arizona              Tucson               1.018333e+05  1.029667e+05   
Oregon               Portland             1.528000e+05  1.547667e+05   
Oklahoma             Oklahoma City        7.643333e+04  7.750000e+04   
Nebraska             Omaha                1.128000e+05  1.141000e+05   
New Mexico           Albuquerque          1.258667e+05  1.267000e+05   
California           Fresno               9.410000e+04  9.526667e+04   
...                                                ...           ...   
Texas                Granite Shoals                NaN           NaN   
Maryland             Piney Point          1.556667e+05  1.551667e+05   
California           Stinson Beach        9.974333e+05  1.120867e+06   
Wisconsin            Maribel                       NaN           NaN   
Idaho                Middleton            1.060667e+05  1.043333e+05   
Kentucky             Almo                 8.403333e+04  8.526667e+04   
Colorado             Bennett              1.329000e+05  1.358333e+05   
New Hampshire        East Hampstead       1.618000e+05  1.691000e+05   
Missouri             Garden City                   NaN           NaN   
Arkansas             Mountainburg         5.716667e+04  6.433333e+04   
Tennessee            Ocoee                5.693333e+04  5.313333e+04   
New York             Upper Brookville     1.230967e+06  1.230967e+06   
Hawaii               Volcano              9.870000e+04  1.053667e+05   
South Carolina       Wedgefield                    NaN           NaN   
Michigan             Williamston          1.591667e+05  1.613000e+05   
Arkansas             Decatur              6.360000e+04  6.440000e+04   
Indiana              Edgewood             9.170000e+04  9.186667e+04   
Tennessee            Palmyra                       NaN           NaN   
Maryland             Saint Inigoes        1.480667e+05  1.476000e+05   
Indiana              Marysville                    NaN           NaN   
Missouri             Bois D Arc           1.078000e+05  1.070000e+05   
Virginia             Henrico              1.285667e+05  1.307667e+05   
New Jersey           Diamond Beach        1.739667e+05  1.831000e+05   
Pennsylvania    

In [13]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    # Prepare data
    start = '2008q2'
    bottom = get_recession_bottom()
    gdp = convert_housing_data_to_quarters()
    gdp = gdp[[start, bottom]]
    
    # (price_ratio=quarter_before_recession/recession_bottom), holy shit! hardcode start to 2008q2
    gdp['ratio'] = gdp[start].div(gdp[bottom])

    
#     # version 1 - left join, to figure out university town and non-univeristy town
#     gdp = gdp.reset_index()
#     university_towns = get_list_of_university_towns()
#     university_towns['marker'] = 1
#     joined = pd.merge(gdp, university_towns, on=['State', 'RegionName'], how='left')
#     university_towns_no = joined[pd.isnull(joined['marker'])]
#     university_towns_no = university_towns_no['ratio']
#     university_towns_no = university_towns_no.dropna()
#     university_towns_yes = joined[joined['marker'] > 0 ]
#     university_towns_yes = university_towns_yes['ratio']
#     university_towns_yes = university_towns_yes.dropna()
#     # version 1 - Result: ('False', 0.028286023029037891, 'university town')
#     # Adjusted Ratio: ('True', 0.0060048246761767012, 'university town')    
 
    
#     ## version 2 -
#     university_towns = get_list_of_university_towns()
#     university_towns = university_towns.set_index(['State', 'RegionName'])    
#     university_towns_yes = gdp.loc[list(university_towns.index)]['ratio'].dropna()
#     university_towns_no_indices = set(gdp.index) - set(university_towns_yes.index)
#     university_towns_no = gdp.loc[list(university_towns_no_indices),:]['ratio'].dropna()
#     # version 2 - Result: ('False', 0.028286023029038057, 'university town')
#     # Adjusted Ratio: ('True', 0.0060048246761772971, 'university town')  ?? slightly difference??


#     ## version 3 - My guess is answer maybe only index on the region, regardless of state? 
#     # version 3 Result: ('True', 1.7330883466678034e-05, 'university town')
#     # Adjusted Ratio: ('True', 7.386925038265953e-06, 'university town')


    # Clinton, New York appears twice in the Housing Data dataframe?? maybe that is the gap of version 1 and 2?
    # https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/FoxE2HGQEeeM7BK4-ybiDA   

    
    # version 5 
    # Go through all the post on the forum, I believe this one is the most important
    #https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/iCSsLhrvEee0thK6z3XwTA
    tuple_list = [tuple(x) for x in get_list_of_university_towns().values]
    university_towns_yes = gdp.loc[tuple_list]
    university_towns_yes = university_towns_yes['ratio']
    university_towns_yes = university_towns_yes.dropna()
    university_towns_no = gdp.loc[~gdp.index.isin(tuple_list)]
    university_towns_no = university_towns_no['ratio']
    university_towns_no = university_towns_no.dropna()
    # version 5 result: (True, 0.0060048246761773031, 'university town')

    # p_val
    p_val = list(ttest_ind(university_towns_no, university_towns_yes))[1]

    # better
    if university_towns_no.mean() > university_towns_yes.mean():
        better = 'university town'
    else:
        better =  'non-university town'
  
    return (p_val < 0.01, p_val, better)

In [14]:
run_ttest()

(True, 0.0060048246761773031, 'university town')